
### Structured output
Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

### Pydantic
Pydantic models provide the richest feature set with field validation, descriptions, and nested structures.

In [2]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model=init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002296C961BD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002296C9625D0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")

In [4]:
model_with_structure=model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002296C961BD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002296C9625D0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', '

In [5]:
model.invoke("Provide details about the moview Inception")

AIMessage(content='<think>\nOkay, so I need to provide details about the movie Inception. Let me start by recalling what I know. Directed by Christopher Nolan, right? He\'s known for complex narratives and high-concept films like Interstellar and The Dark Knight. Inception was released in 2010, I think. The main cast includes Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, and others. The story is about entering dreams to plant or extract ideas. The concept is called "inception," which is the act of implanting an idea in someone\'s subconscious.\n\nThe plot involves Dom Cobb, played by DiCaprio, a professional thief who steals information by infiltrating the subconscious of his targets. He\'s offered a chance to erase his criminal past by performing the reverse: planting an idea into someone\'s mind instead. The team of experts he assembles includes Arthur (Tom Hardy), who handles the practical aspects, Ariadne (Ellen Page), an architect who designs the dream spaces, an

In [6]:
response=model_with_structure.invoke("Provide details about the moview Inception")
response

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

In [1]:
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq

# 1. Define the 'Blueprint'
class NewsArticle(BaseModel):
    headline: str = Field(description="The main title of the news")
    sentiment: str = Field(description="Is the news positive, negative, or neutral?")
    impact_score: int = Field(description="Score from 1-10 on importance")

# 2. Bind it to the model
model = ChatGroq(model="llama-3.3-70b-versatile")
structured_llm = model.with_structured_output(NewsArticle)

# 3. Invoke - the result is a Python Object, not a string!
result = structured_llm.invoke("Anthropic released a new safety framework today.")

print(f"Headline: {result.headline}")
print(f"Sentiment: {result.sentiment}")
print(f"Impact Score: {result.impact_score}")

Headline: Anthropic Releases New Safety Framework
Sentiment: positive
Impact Score: 8
